in this project we are trying to generate a inteligent agent that can sees what will happen if it make a sequence of moves of limited depth, and also we are trying to find a heuristic function which guesses how much a state is good or bad for us to to be in. in this way we maximize our chance of winning the whole game by making good moves in each state and move toward better states.




In [ ]:
import random
import copy
import math
import time


class GameError(AttributeError):
    pass


class Game:

    def __init__(self, n):
        self.size = n
        self.half_the_size = int(n/2)
        self.reset()

    def reset(self):
        self.board = []
        value = 'B'
        for i in range(self.size):
            row = []
            for j in range(self.size):
                row.append(value)
                value = self.opponent(value)
            self.board.append(row)
            if self.size % 2 == 0:
                value = self.opponent(value)

    def __str__(self):
        result = "  "
        for i in range(self.size):
            result += str(i) + " "
        result += "\n"
        for i in range(self.size):
            result += str(i) + " "
            for j in range(self.size):
                result += str(self.board[i][j]) + " "
            result += "\n"
        return result

    def valid(self, row, col):
        return row >= 0 and col >= 0 and row < self.size and col < self.size

    def contains(self, board, row, col, symbol):
        return self.valid(row, col) and board[row][col] == symbol

    def countSymbol(self, board, symbol):
        count = 0
        for r in range(self.size):
            for c in range(self.size):
                if board[r][c] == symbol:
                    count += 1
        return count

    def opponent(self, player):
        if player == 'B':
            return 'W'
        else:
            return 'B'

    def distance(self, r1, c1, r2, c2):
        return abs(r1-r2 + c1-c2)

    def makeMove(self, player, move):
        self.board = self.nextBoard(self.board, player, move)

    def nextBoard(self, board, player, move):
        r1 = move[0]
        c1 = move[1]
        r2 = move[2]
        c2 = move[3]
        next = copy.deepcopy(board)
        if not (self.valid(r1, c1) and self.valid(r2, c2)):
            raise GameError
        if next[r1][c1] != player:
            raise GameError
        dist = self.distance(r1, c1, r2, c2)
        if dist == 0:
            if self.openingMove(board):
                next[r1][c1] = "."
                return next
            raise GameError
        if next[r2][c2] != ".":
            raise GameError
        jumps = int(dist/2)
        dr = int((r2 - r1)/dist)
        dc = int((c2 - c1)/dist)
        for i in range(jumps):
            if next[r1+dr][c1+dc] != self.opponent(player):
                raise GameError
            next[r1][c1] = "."
            next[r1+dr][c1+dc] = "."
            r1 += 2*dr
            c1 += 2*dc
            next[r1][c1] = player
        return next

    def openingMove(self, board):
        return self.countSymbol(board, ".") <= 1

    def generateFirstMoves(self, board):
        moves = []
        moves.append([0]*4)
        moves.append([self.size-1]*4)
        moves.append([self.half_the_size]*4)
        moves.append([(self.half_the_size)-1]*4)
        return moves

    def generateSecondMoves(self, board):
        moves = []
        if board[0][0] == ".":
            moves.append([0, 1]*2)
            moves.append([1, 0]*2)
            return moves
        elif board[self.size-1][self.size-1] == ".":
            moves.append([self.size-1, self.size-2]*2)
            moves.append([self.size-2, self.size-1]*2)
            return moves
        elif board[self.half_the_size-1][self.half_the_size-1] == ".":
            pos = self.half_the_size - 1
        else:
            pos = self.half_the_size
        moves.append([pos, pos-1]*2)
        moves.append([pos+1, pos]*2)
        moves.append([pos, pos+1]*2)
        moves.append([pos-1, pos]*2)
        return moves

    def check(self, board, r, c, rd, cd, factor, opponent):
        if self.contains(board, r+factor*rd, c+factor*cd, opponent) and \
           self.contains(board, r+(factor+1)*rd, c+(factor+1)*cd, '.'):
            return [[r, c, r+(factor+1)*rd, c+(factor+1)*cd]] + \
                self.check(board, r, c, rd, cd, factor+2, opponent)
        else:
            return []

    def generateMoves(self, board, player):
        if self.openingMove(board):
            if player == 'B':
                return self.generateFirstMoves(board)
            else:
                return self.generateSecondMoves(board)
        else:
            moves = []
            rd = [-1, 0, 1, 0]
            cd = [0, 1, 0, -1]
            for r in range(self.size):
                for c in range(self.size):
                    if board[r][c] == player:
                        for i in range(len(rd)):
                            moves += self.check(board, r, c, rd[i], cd[i], 1,
                                                self.opponent(player))
            return moves

    def playOneGame(self, p1, p2, show):
        self.reset()
        while True:
            if show:
                print(self)
                print("player B's turn")
            move = p1.getMove(self.board)
            if move == []:
                print("Game over")
                return 'W'
            try:
                self.makeMove('B', move)
            except GameError:
                print("Game over: Invalid move by", p1.name)
                print(move)
                print(self)
                return 'W'
            if show:
                print(move)
                print(self)
                print("player W's turn")
            move = p2.getMove(self.board)
            if move == []:
                print("Game over")
                return 'B'
            try:
                self.makeMove('W', move)
            except GameError:
                print("Game over: Invalid move by", p2.name)
                print(move)
                print(self)
                return 'B'
            if show:
                print(move)

    def playNGames(self, n, p1, p2, show):
        first = p1
        second = p2
        for i in range(n):
            print("Game", i)
            winner = self.playOneGame(first, second, show)
            if winner == 'B':
                first.won()
                second.lost()
                print(first.name, "wins")
            else:
                first.lost()
                second.won()
                print(second.name, "wins")
            first, second = second, first


class Player:
    name = "Player"
    wins = 0
    losses = 0

    def results(self):
        result = self.name
        result += " Wins:" + str(self.wins)
        result += " Losses:" + str(self.losses)
        return result

    def lost(self):
        self.losses += 1

    def won(self):
        self.wins += 1

    def reset(self):
        self.wins = 0
        self.losses = 0

    def initialize(self, side):
        abstract()

    def getMove(self, board):
        abstract()


class SimplePlayer(Game, Player):
    def initialize(self, side):
        self.side = side
        self.name = "Simple"

    def getMove(self, board):
        moves = self.generateMoves(board, self.side)
        n = len(moves)
        if n == 0:
            return []
        else:
            return moves[0]


class RandomPlayer(Game, Player):
    def initialize(self, side):
        self.side = side
        self.name = "Random"

    def getMove(self, board):
        moves = self.generateMoves(board, self.side)
        n = len(moves)
        if n == 0:
            return []
        else:
            return moves[random.randrange(0, n)]


class HumanPlayer(Game, Player):
    def initialize(self, side):
        self.side = side
        self.name = "Human"

    def getMove(self, board):
        moves = self.generateMoves(board, self.side)
        while True:
            print("Possible moves:", moves)
            n = len(moves)
            if n == 0:
                print("You must concede")
                return []
            index = input("Enter index of chosen move (0-" + str(n-1) +
                          ") or -1 to concede: ")
            try:
                index = int(index)
                if index == -1:
                    return []
                if 0 <= index <= (n-1):
                    print("returning", moves[index])
                    return moves[index]
                else:
                    print("Invalid choice, try again.")
            except Exception as e:
                print("Invalid choice, try again.")


# Sequence of moves

first we are implementing the Minimax Agent. this agent has to watch for its deeper states which are the states that agent will come up in them if it make some particular sequence of moves. 

---

# Switchin between maximizer and minimizer agents

also its good to point out the fact that in each depth we are switchin between the agent who is trying to maximize its chance of winning and the agent who is trying to minimize its opponent's chance of winning.


---
# Introduction

in this part we are explaining what each agent does for its target to whether maximize its chance of winning or minimize its opponent'c chance of winning .

---
# Calling your opponent to minimize your chance of winning

in each step or depth agents calculate what moves they can make and after that they check if that move is maximizing their chance of wining or not and also they are doing this with a way similar to dfs cause they themselves call the other agent that is their opponent to see what it will do too.

---

# Limited Depth for better timing

we want to determine what is our best move in the current state in a short time so we just check the states in the state space of our problem with a limited depth. this will make every move to take at most $5 seconds$.

---

# Why we have to guess what coming ?

as we can not see what will comes up in limited moves, so we have to guess what will come in the deeper states so we need a function to calculate our chance of winning given a special board.

---
# Heuristic (evaluation function)

we want a function to take a board and guess how much this board is likely to be a better situation for your agent by just looking at the details of the board you have right now.

for this purpose we are using the moves that agent can do and also the counts of agent in the map board .

and because we want to maximze our chance of winning and also minimize our opponents chance we subtract these measures for our agents to determine which move is the best move that we can do now.


the other reason we are choosing this heuristic function among others is that in every step is important that how many moves we can make in that state cause the whole game is about having the option of moving in every step and trying to survive this way.

another thing that is included in our model is that we are checking how many of our side nuts is in the map board in that state and this is important cause with some intution about the game; if in any state we have more nuts than our opponent it is more likely that we can win that game.


In [ ]:
class MinimaxPlayer(Game, Player):

    def __init__(self, n, depth):
        super().__init__(n)
        self.depth = depth
        self.totalTime = 0
        self.moveCounts = 0

    def initialize(self, side):
        self.side = side
        self.name = "minimax"
    
    def getAverageMoveTime(self):
        return self.totalTime / self.moveCounts

    def getMove(self, board):
        self.moveCounts += 1
        startTime = time.time()
        moves = self.generateMoves(board, self.side)
        if len(moves) == 0:
            return []
        maxVal = -math.inf
        maxIndex = -1
        for index, move in enumerate(moves):
            newBoard = self.nextBoard(board, self.side, move)
            val = self.minPlayer(newBoard, self.depth)
            if val > maxVal:
                maxVal = val
                maxIndex = index
        if maxIndex == -1:
            return []
        finishTime = time.time()
        self.totalTime += finishTime - startTime
        print(finishTime - startTime)
        return moves[maxIndex]

    def maxPlayer(self, board, depth):
        moves = self.generateMoves(board, self.side)
        if depth == 0 or len(moves) == 0:
            return self.calcHeu(board)
        val = -math.inf
        for m in moves:
            newBoard = self.nextBoard(board, self.side, m)
            val = max(val, self.minPlayer(newBoard, depth - 1))
        return val

    def minPlayer(self, board, depth):
        moves = self.generateMoves(board, self.opponent(self.side))
        if depth == 0 or len(moves) == 0:
            return self.calcHeu(board)
        val = math.inf
        for m in moves:
            newBoard = self.nextBoard(board, self.opponent(self.side), m)
            val = min(val, self.maxPlayer(newBoard, depth - 1))
        return val

    def calcHeu(self, board):
        mine = 0
        other = 0
        for r in range(self.size):
            for c in range(self.size):
                if board[r][c] == self.side:
                    mine += 1
                elif board[r][c] == self.opponent(self.side):
                    other += 1
        return ( mine + len(self.generateMoves(board, self.side)) ) - ( other  + len(self.generateMoves(board, self.opponent(self.side))))

# Minimax Player with Alpha Beta Pruning

in this part we use a special type of pruning called alpha beta pruning .

in this approach we use two values to check if we have to traverse all the graph and moves that we have or not.
we can determine this by checking what is our best value we can get from origin of some sequence of moves and check if the children of that node are likely to give us better options or not.
if they can not give us better options we can just prune them.

except this pruning approach that we use the rest of the actions we do in every step is similar to previous version of Minimax Player.


In [ ]:
class MinimaxPrunerPlayer(Game, Player):

    def __init__(self, n, depth):
        super().__init__(n)
        self.depth = depth
        self.totalTime = 0
        self.moveCounts = 0

    def initialize(self, side):
        self.side = side
        self.name = "minimax"
        

    def getAverageMoveTime(self):
        return self.totalTime / self.moveCounts

    def getMove(self, board):
        self.moveCounts += 1
        startTime = time.time()
        alpha = -math.inf
        beta = math.inf
        moves = self.generateMoves(board, self.side)
        if len(moves) == 0:
            return []
        maxVal = -math.inf
        maxIndex = -1
        for index, move in enumerate(moves):
            newBoard = self.nextBoard(board, self.side, move)
            val = self.minPrunerPlayer(newBoard, self.depth, alpha, beta)
            if val > maxVal:
                maxVal = val
                maxIndex = index
            if val > beta:
                return val
            alpha = max(alpha, val)
        if maxIndex == -1:
            return []
        finishTime = time.time()
        self.totalTime += finishTime - startTime
        print(finishTime - startTime)
        return moves[maxIndex]

    def maxPrunerPlayer(self, board, depth, alpha, beta):
        moves = self.generateMoves(board, self.side)
        if depth == 0 or len(moves) == 0:
            return self.calcHeu(board)
        val = -math.inf
        for m in moves:
            newBoard = self.nextBoard(board, self.side, m)
            val = max(val, self.minPrunerPlayer(newBoard, depth - 1, alpha, beta))
            if val > beta:
                return val
            alpha = max(alpha, val)
        return val

    def minPrunerPlayer(self, board, depth, alpha, beta):
        moves = self.generateMoves(board, self.opponent(self.side))
        if depth == 0 or len(moves) == 0:
            return self.calcHeu(board)
        val = math.inf
        for m in moves:
            newBoard = self.nextBoard(board, self.opponent(self.side), m)
            val = min(val, self.maxPrunerPlayer(newBoard, depth - 1, alpha, beta))
            if val < alpha:
                return val
            beta = min(beta, val)
        return val

    def calcHeu(self, board):
        mine = 0
        other = 0
        for r in range(self.size):
            for c in range(self.size):
                if board[r][c] == self.side:
                    mine += 1
                elif board[r][c] == self.opponent(self.side):
                    other += 1
        return ( mine + len(self.generateMoves(board, self.side)) ) - ( other  + len(self.generateMoves(board, self.opponent(self.side))))


# Is there any difference between Minimax Agent and Alpha beta Agent moves ?

there is no difference in their moves cause in every step we are doing the same thing in both agents , the only thing that is missing in the Normal Minimax Agent is that we are not pruning the leaves and moves that we are sure that they are not our best move to make.
but in alpha beta pruning we identify those pathes and we just prune them in our graph. this way we are optimizing the time that is taken in each state to determine which is the best move here.

In [ ]:
if __name__ == '__main__':
    game = Game(8)
    random1 = MinimaxPlayer(8, 3)
    random1.initialize('B')
    random2 = MinimaxPlayer(8, 2)
    random2.initialize('W')
    game.playOneGame(random1, random2, True)    
    print("the average time: ", random1.getAverageMoveTime())

# Minimax Player 

$4 \times 4 $ board

depth: 3                  
average time for moves: 0.0043512071881975445 seconds

$8 \times 8 $ board

depth: 3

average time for moves: 1.6297348478566045 seconds

$4 \times 4 $ board

depth: 4                  
average time for moves: 0.008295774459838867 seconds

$8 \times 8 $ board

depth: 4

average time for moves: 23.608197584152222 seconds

# Alpha Beta Pruner

$4 \times 4 $ board

depth: 3                  
average time for moves: 0.002435718263898577 seconds

$8 \times 8 $ board

depth: 3

average time for moves: 0.3941378489784572 seconds

$4 \times 4 $ board

depth: 4                  
average time for moves: 0.005474408467610677 seconds

$8 \times 8 $ board

depth: 4

average time for moves: 2.039293031692505 seconds